# Random Forest Model

The objective of this model is to generate predictions using a model for each coupling type. The type of model used is a random forest with 50 trees and with a slight hyperparameter tuning. The score achieved with this model is -0.848 on Kaggle.

In [12]:
import pandas as pd
import numpy as np
from magnetic_interactions import data_utils as du

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

from joblib import dump

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Import and Feature Generation

The following cell joins in a single dataframe all the features generated for the challenge. The features are loaded from previously created csv files for time purposes.

The result of running the cell are dictionaries with a dataframe for each one of the eight coupling types.

In [15]:
train_data_path = './data/champs-scalar-coupling/train.csv'
test_data_path = './data/champs-scalar-coupling/test.csv'

structures_data_path = './data/champs-scalar-coupling/structures.csv'

angles_torsions_train_path = './data/champs-scalar-coupling/angles_torsions_train.csv'
angles_torsions_test_path = './data/champs-scalar-coupling/angles_torsions_test.csv'
bonds_train_path = './data/champs-scalar-coupling/bonds_train.csv'
bonds_test_path = './data/champs-scalar-coupling/bonds_test.csv'

distances_train_path = './data/champs-scalar-coupling/distance_features_train.csv'
distances_test_path = './data/champs-scalar-coupling/distance_features_test.csv'

spins_train_path = './data/champs-scalar-coupling/spin_feature_train.csv'
spins_test_path = './data/champs-scalar-coupling/spin_feature_test.csv'

hyb_train_path = './data/champs-scalar-coupling/hyb_train.csv'
hyb_test_path = './data/champs-scalar-coupling/hyb_test.csv'

features_camilo_train_path = './data/champs-scalar-coupling/features_camilo_train.csv'
features_camilo_test_path = './data/champs-scalar-coupling/features_camilo_test.csv'

train_dfs_dict, targets_dict = du.data_pipeline(train_data_path, structures_data_path, 
                                                angles_torsions_train_path, bonds_train_path, 
                                                distances_train_path, spins_train_path, hyb_train_path,
                                                features_camilo_train_path,
                                                train_data=True)
test_dfs_dict, _ = du.data_pipeline(test_data_path, structures_data_path, angles_torsions_test_path, 
                                    bonds_test_path, distances_test_path, spins_test_path, hyb_test_path,
                                    features_camilo_test_path,
                                    train_data=False)

Mem. usage decreased to 106.62 Mb (50.0% reduction)
Mem. usage decreased to 51.74 Mb (52.1% reduction)
Mem. usage decreased to 279.87 Mb (53.3% reduction)
Mem. usage decreased to 342.06 Mb (33.0% reduction)
Mem. usage decreased to 350.95 Mb (15.1% reduction)
Mem. usage decreased to 364.27 Mb (13.7% reduction)
Mem. usage decreased to 852.93 Mb (63.2% reduction)
Mem. usage decreased to 128.54 Mb (0.0% reduction)
Mem. usage decreased to 68.50 Mb (0.0% reduction)
Mem. usage decreased to  7.86 Mb (0.0% reduction)
Mem. usage decreased to 21.61 Mb (0.0% reduction)
Mem. usage decreased to 206.69 Mb (0.0% reduction)
Mem. usage decreased to 107.02 Mb (0.0% reduction)
Mem. usage decreased to 273.68 Mb (0.0% reduction)
Mem. usage decreased to 30.15 Mb (0.0% reduction)
Mem. usage decreased to 52.57 Mb (45.0% reduction)
Mem. usage decreased to 51.74 Mb (52.1% reduction)
Mem. usage decreased to 145.76 Mb (54.1% reduction)
Mem. usage decreased to 179.21 Mb (33.6% reduction)
Mem. usage decreased to 183

/home/carlosmiguelpatino/Lacuna/projects/MagneticInteractionsChallenge/magnetic_interactions/data_utils.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype(np.int32)
/home/carlosmiguelpatino/Lacuna/projects/MagneticInteractionsChallenge/magnetic_interactions/data_utils.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype(np.float16)


Mem. usage decreased to 111.10 Mb (0.0% reduction)
Mem. usage decreased to 68.97 Mb (0.0% reduction)
Mem. usage decreased to 57.52 Mb (0.0% reduction)
Mem. usage decreased to 147.13 Mb (0.0% reduction)
Mem. usage decreased to 36.81 Mb (0.0% reduction)
Mem. usage decreased to  4.38 Mb (0.0% reduction)
Mem. usage decreased to 16.42 Mb (0.0% reduction)
Mem. usage decreased to 11.67 Mb (0.0% reduction)


## Feature Selection

Select which features are going to be used for training the model for each type.

In [16]:
features_coupling_type = {
    
    '1JHC': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
            'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N',
            'n_bonds_mol', 'n_bonds_0', 'n_bonds_1', 
            'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb', 
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1'],
    '2JHH': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
            'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N', 'angles',
            'n_bonds_mol', 'n_bonds_0', 'n_bonds_1',
            'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb',
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1'],
    '1JHN': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
            'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N',
            'n_bonds_mol', 'n_bonds_0', 'n_bonds_1',
            'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb',
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1'],
    '2JHN': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
            'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N', 'angles',
            'n_bonds_mol', 'n_bonds_0', 'n_bonds_1',
            'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb',
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1'],
    '2JHC': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
            'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N', 'angles',
            'n_bonds_mol', 'n_bonds_0', 'n_bonds_1',
            'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb',
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1'],
    '3JHH': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
             'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N',
             'torsions', 'n_bonds_mol', 'n_bonds_0', 'n_bonds_1', 'karplus_1',
             'karplus_2', 'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb',
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1'], 
    '3JHC': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
             'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N',
             'torsions', 'n_bonds_mol', 'n_bonds_0', 'n_bonds_1', 'karplus_1',
             'karplus_2', 'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb',
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1'], 
    '3JHN': ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'n_atoms', 'n_C', 'n_F',
             'n_H', 'n_N', 'n_O', 'type_1__C', 'type_1__H', 'type_1__N',
             'torsions', 'n_bonds_mol', 'n_bonds_0', 'n_bonds_1', 'karplus_1',
             'karplus_2', 'max_distance_mol' , 'mean_distance_mol', 'dist','1/r2',
            'spin', 'n_hyb_3', 'hyb_coupled_atom', '1/hyb',
            'volume', 'max diag', 'median diag', 'min diag',
       'distance_atomic_weight_0', 'distance_en_allen_0',
       'distance_atomic_weight_1', 'distance_en_allen_1', 'C_0d_0', 'N_0d_0',
       'O_0d_0', 'C_1d_0', 'H_1d_0', 'N_1d_0', 'O_1d_0', 'C_2d_0', 'F_2d_0',
       'H_2d_0', 'N_2d_0', 'O_2d_0', 'C_3d_0', 'F_3d_0', 'H_3d_0', 'N_3d_0',
       'O_3d_0', 'C_4d_0', 'F_4d_0', 'H_4d_0', 'N_4d_0', 'O_4d_0', 'C_0d_1',
       'F_0d_1', 'H_0d_1', 'N_0d_1', 'O_0d_1', 'C_1d_1', 'F_1d_1', 'H_1d_1',
       'N_1d_1', 'O_1d_1', 'C_2d_1', 'F_2d_1', 'H_2d_1', 'N_2d_1', 'O_2d_1',
       'C_3d_1', 'F_3d_1', 'H_3d_1', 'N_3d_1', 'O_3d_1', 'C_4d_1', 'F_4d_1',
       'H_4d_1', 'N_4d_1', 'O_4d_1']
    
}

## Tuned Hyperparameters

Create a dictionary for the hyperparameters found in a small hyperparameter tuning for each model.

In [17]:
best_params = {'1JHC': {'bootstrap': False,
  'max_depth': None,
  'max_features': 6,
  'min_samples_split': 10},
 '2JHH': {'bootstrap': False,
  'max_depth': None,
  'max_features': 8,
  'min_samples_split': 4},
 '1JHN': {'bootstrap': False,
  'max_depth': None,
  'max_features': 8,
  'min_samples_split': 4},
 '2JHN': {'bootstrap': False,
  'max_depth': None,
  'max_features': 8,
  'min_samples_split': 4},
 '2JHC': {'bootstrap': False,
  'max_depth': None,
  'max_features': 6,
  'min_samples_split': 10},
 '3JHH': {'bootstrap': False,
  'max_depth': None,
  'max_features': 8,
  'min_samples_split': 4},
 '3JHC': {'bootstrap': False,
  'max_depth': None,
  'max_features': 8,
  'min_samples_split': 4},
 '3JHN': {'bootstrap': False,
  'max_depth': None,
  'max_features': 8,
  'min_samples_split': 4}}

## Fit Models and Generate Predictions

In [21]:
submission_df = pd.DataFrame()
train_predictions = pd.DataFrame()
keys = list(train_dfs_dict.keys())

for i, key in enumerate(keys):
    
    train_df = train_dfs_dict[key]
    test_df = test_dfs_dict[key]
    y_train = targets_dict[key]
    features = features_coupling_type[key]  
    
    train_pred_df = pd.DataFrame()
    test_pred_df = pd.DataFrame()
    train_pred_df['id'] = train_df['id']
    test_pred_df['id'] = test_df['id']
    
    X_train = train_df[features]
    X_test = test_df[features]
    
    print('Fitting {0} samples for {1} ...'.format(len(train_pred_df), key))
    
    tuned_params = best_params[key]
    rf_reg = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=2, verbose=2, **tuned_params)
    rf_reg.fit(X_train, y_train)
    dump(rf_reg, './models_win/rf_reg_{}.joblib'.format(key))
    y_hat_train = rf_reg.predict(X_train)
    y_hat_test = rf_reg.predict(X_test)
    
    print('Finished training! {}/8'.format(i+1))
    
    train_pred_df['scalar_coupling_constant'] = y_hat_train
    test_pred_df['scalar_coupling_constant'] = y_hat_test
    train_predictions = train_predictions.append(train_pred_df)
    submission_df = submission_df.append(test_pred_df)
    
    score = mean_absolute_error(y_train, y_hat_train)
    
    print('Train MAE:', score, '\n')
    
    del test_dfs_dict[key], train_dfs_dict[key]

Fitting 709416 samples for 1JHC ...


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 50building tree 2 of 50

building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  1.3min


building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


...

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   26.2s


building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:   37.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.8s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    2.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    1.4s finished


Finished training! 8/8
Train MAE: 0.018588197192906594 



In [6]:
submission_df = submission_df.sort_values('id')
submission_df.to_csv('./submissions/neighbors_features.csv', index=False)
train_predictions.to_csv('./data/predicted_couplings.csv')